| Technological Institute of the Philippines | Quezon City - Computer Engineering                                      |
| ------------------------------------------ | ----------------------------------------------------------------------- |
| Course Code:                               | CPE 313                                                                 |
| Code Title:                                | Advanced Machine Learning and Deep Learning                             |
| 1st Semester                               | AY 2023-2024                                                            |
| <hr>                                       | <hr>                                                                    |
| <u>**ACTIVITY NO.**                        | **Performing Face Recognition**                                         |
| **Name**                                   | Estacion, Kenneth                                                       |
| **Section**                                | CPE32S8                                                                 |
| **Date Performed**:                        | Feb. 20, 2024                                                           |
| **Date Submitted**:                        | Feb. 20, 2024                                                           |
| **Instructor**:                            | Dr. Jonathan V. Taylar / Engr. Verlyn V. Nojor / Engr. Roman M. Richard |

<hr>


## 1. Objectives


This activity aims to enable students to perform data preparation and face recognition on their own generated dataset.


## 2. Intended Learning Outcomes (ILOs)

After this activity, the students should be able to:

- Utilize data preparation techniques for images.
- Perform Face Recognition using multiple algorithms.
- Evaluate the performance of different algorithms.


## 3. Procedures and Outputs


### Preparing the training data


Now that we have our data, we need to load these sample pictures into our face recognition algorithms. All face recognition algorithms take two parameters in their `train()` method: an array of images and an array of labels. What do these labels represent? They are the IDs of a certain individual/face so that when face recognition is performed, we not only know the person was recognized but also who—among the many people available in our database—the person is.

To do that, we need to create a comma-separated value (CSV) file, which will contain the path to a sample picture followed by the ID of that person.


**Include a Screenshot of Your Dataset Here**


---


### Loading the data and recognizing faces


Next up, we need to load these two resources (the array of images and CSV file) into the face recognition algorithm, so it can be trained to recognize our face. To do this, we build a function that reads the CSV file and—for each line of the file—loads the image at the corresponding path into the images array and the ID into the labels array.


In [1]:
import numpy as np
import os
import errno
import sys
import cv2

def read_images(path, sz=None):
  c = 0
  X, y = [], []

  for dirname, dirnames, filenames in os.walk(path):
    for subdirname in dirnames:
      subject_path = os.path.join(dirname, subdirname)
      for filename in os.listdir(subject_path):
        try:
          if(filename == ".directory"):
            continue
          filepath = os.path.join(subject_path, filename)
          im = cv2.imread(os.path.join(subject_path, filename), cv2.IMREAD_GRAYSCALE)

          # Resize the images to the prescribed size
          if (sz is not None):
            im = cv2.resize(im, (200,200))

          X.append(np.asarray(im, dtype=np.uint8))
          y.append(c)

        except IOError as e:
          print(f"I/O Error({e.errno}): {e.strerror}")
        except:
          print("Unexpected error:", sys.exc_info()[0])
          raise
      c = c+1
  return [X, y]

In [2]:
path = 'images'
[X,y] = read_images(path,sz=200)

**Question: Run the function above on your generated dataset. Provide an analysis and note all the challenges you have encountered running this code.**


---


### Performing Face Recognition Algorithms


Here is a sample script for testing the Face Recognition Algorithm. In this section, we're going to follow the same process but with different algorithms for face recognitions, namely:

- Eigenface Recognition
- Fisherface Recognition
- Local Binary Pattern Histograms (LBPH) Recognition


In [3]:
names = ['Ed','Jericho','Jose','Kenneth','Marie']
for i in range(5):
    print(i,':',y.count(i))
y = np.asarray(y, dtype=np.int32)
print(y)

0 : 17
1 : 33
2 : 7
3 : 23
4 : 15
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


In [4]:
def face_rec():
  if len(sys.argv) < 2:
    print("USAGE: facerec_demo.py </path/to/images> [</path/to/store/images/at>]")
    sys.exit()

  [X,y] = read_images(path,sz=200)
  y = np.asarray(y, dtype=np.int32)
  model = cv2.face.EigenFaceRecognizer_create()
  model.train(X, y)

  camera = cv2.VideoCapture(0)
  face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

  while True:
    ret, img = camera.read()
    if not ret:
      break

    faces = face_cascade.detectMultiScale(img, 1.5, 5)

    for (x, y, w, h) in faces:
      cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
      gray = cv2.cvtColor(img[y:y + h, x:x + w], cv2.COLOR_BGR2GRAY)
      roi = cv2.resize(gray, (200, 200), interpolation=cv2.INTER_LINEAR)

      try:
        params = model.predict(roi)
        label = names[params[0]]
        cv2.putText(img, label + ", " + str(params[1]), (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
      except:
        continue

    cv2.imshow("camera", img)
    if cv2.waitKey(1) & 0xFF == ord("q"):
      break

  camera.release()
  cv2.destroyAllWindows()

if __name__ == "__main__":
  face_rec()


**Question: Provide an analysis of the sample script for the process using the Eigenface Model. What is the sample code doing? Are you able to troubleshoot any problems encountered?**


---

Perform the remaining face recognition techniques by using the same (or modified) process from the sample code:

- `model = cv2.face.createFisherFaceRecognizer()`
- `model = cv2.face.createLBPHFaceRecognizer()`


In [5]:
#model = cv2.face.createLBPHFaceRecognizer()
def face_rec():
  if len(sys.argv) < 2:
    print("USAGE: facerec_demo.py </path/to/images> [</path/to/store/images/at>]")
    sys.exit()

  [X, y] = read_images(path,sz=200)
  y = np.asarray(y, dtype=np.int32)

  model = cv2.face.LBPHFaceRecognizer_create()
  model.train(X, y)

  camera = cv2.VideoCapture(0)
  face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

  while True:
    ret, img = camera.read()
    if not ret:
      break

    faces = face_cascade.detectMultiScale(img, 1.3, 5)

    for (x, y, w, h) in faces:
      cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
      gray = cv2.cvtColor(img[y:y + h, x:x + w], cv2.COLOR_BGR2GRAY)
      roi = cv2.resize(gray, (200, 200), interpolation=cv2.INTER_LINEAR)

      try:
        params = model.predict(roi)
        label = names[params[0]]
        cv2.putText(img, label + ", " + str(params[1]), (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
      except:
        continue

    cv2.imshow("camera", img)
    if cv2.waitKey(1) & 0xFF == ord("q"):
      break

  camera.release()
  cv2.destroyAllWindows()

if __name__ == "__main__":
  face_rec()


In [35]:
#model = cv2.face.createFisherFaceRecognizer()`
def face_rec():
  if len(sys.argv) < 2:
    print("USAGE: facerec_demo.py </path/to/images> [</path/to/store/images/at>]")
    sys.exit()

  [X, y] = read_images(path,sz=200)
  y = np.asarray(y, dtype=np.int32)

  model = cv2.face.FisherFaceRecognizer.create()
  model.train(X, y)

  camera = cv2.VideoCapture(0)
  face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

  while True:
    ret, img = camera.read()
    if not ret:
      break

    faces = face_cascade.detectMultiScale(img, 1.3, 5)

    for (x, y, w, h) in faces:
      cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
      gray = cv2.cvtColor(img[y:y + h, x:x + w], cv2.COLOR_BGR2GRAY)
      roi = cv2.resize(gray, (200, 200), interpolation=cv2.INTER_LINEAR)

      try:
        params = model.predict(roi)
        label = names[params[0]]
        cv2.putText(img, label + ", " + str(params[1]), (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
      except:
        continue

    cv2.imshow("camera", img)
    if cv2.waitKey(1) & 0xFF == ord("q"):
      break

  camera.release()
  cv2.destroyAllWindows()

if __name__ == "__main__":
  face_rec()


**Question: The `predict()` method returns a two-element array. Provide your analysis of the two returned values and their important ince this application.**


## 4. Supplementary Activity


Your accomplisment of the tasks below contribute to the achievement of ILO1, ILO2, and ILO3 for this module.

---

Tasks:

1. Create a new dataset for testing, this dataset must include the following:

- The same person/s that the model has to recognize.
- Different person/s that the model should not recognize.

2. For each model, perform 20 tests. Document the testing performed and provide observations.
3. Conclude on the performed tests by providing your evaluation of the performance of the models.


In [38]:
[X,y] = read_images(path,sz=200)
X




































































































































































































































































[array([[192, 189, 186, ..., 133, 152, 154],
        [187, 185, 184, ..., 162, 158, 163],
        [186, 184, 173, ..., 159, 158, 161],
        ...,
        [197, 195, 194, ..., 173, 175, 176],
        [193, 196, 190, ..., 174, 174, 177],
        [193, 197, 198, ..., 169, 172, 172]], dtype=uint8),
 array([[179, 182, 172, ..., 137, 162, 147],
        [179, 169, 145, ..., 171, 157, 155],
        [166, 125,  75, ..., 161, 162, 158],
        ...,
        [195, 195, 190, ..., 175, 178, 179],
        [197, 195, 193, ..., 175, 179, 178],
        [192, 199, 196, ..., 174, 177, 180]], dtype=uint8),
 array([[174, 236, 228, ..., 243, 239, 226],
        [231, 158, 230, ..., 236, 232, 237],
        [229, 169, 227, ..., 255, 255, 232],
        ...,
        [162, 158, 141, ..., 179, 181, 178],
        [159, 150, 132, ..., 179, 182, 182],
        [151, 139, 131, ..., 182, 180, 175]], dtype=uint8),
 array([[194, 192, 195, ..., 213, 212, 209],
        [190, 193, 195, ..., 219, 214, 210],
        [190, 19

In [37]:
import pandas as pd
df = pd.DataFrame([X,y]).T
df.columns = ['X','y']
df[y] = df[y].astype(int)
recognize = df[df[y].isin([0,1])]
not_recognize = df[df[y].isin([2,3,4])]
df

KeyError: "None of [Index([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,\n       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,\n       1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,\n       3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],\n      dtype='int32')] are in the [columns]"

In [20]:
not_recognize[1].value_counts()

1
3    23
4    15
2     7
Name: count, dtype: int64

In [21]:
recognize[1].value_counts()

1
1    33
0    17
Name: count, dtype: int64

In [32]:
not_recognize_x = np.array(not_recognize.)
not_recognize_y = np.array(not_recognize[1])
recognize_x = np.array(recognize[0])
recognize_y = np.array(recognize[1])

In [33]:
not_recognize_x

array([array([[255, 255, 255, ..., 255, 255, 255],
              [255, 255, 255, ..., 255, 255, 255],
              [255, 255, 255, ..., 255, 255, 255],
              ...,
              [ 43,  66,  74, ...,  28,  12,   0],
              [ 27,  56,  72, ...,  21,   0,  14],
              [ 21,  40,  71, ...,   9,   0,  16]], dtype=uint8), 2],
      dtype=object)

## 5. Summary, Conclusions and Lessons Learned


<hr/>

**_Proprietary Clause_**

_Property of the Technological Institute of the Philippines (T.I.P.). No part of the materials made and uploaded in this learning management system by T.I.P. may be copied, photographed, printed, reproduced, shared, transmitted, translated, or reduced to any electronic medium or machine-readable form, in whole or in part, without the prior consent of T.I.P._
